In [4]:
import pandas as pd
import re

# Les CSV
df = pd.read_csv("../data/mozart_catalog_rows.csv")  # juster path hvis nødvendig

# Fjern eksisterende id-kolonne hvis den finnes
if "id" in df.columns:
    df = df.drop(columns=["id"])

def extract_k9_key2(k):
    """Trekk ut heltall fra starten av k9_key.
       - stopper ved første ikke-siffer (dvs. '1,02' -> 1)
       - 'Anh...' -> 9999
       - NaN -> 9999
    """
    if pd.isna(k):
        return 9999

    s = str(k).strip()

    # 'Anh' håndteres som 9999
    if s.lower().startswith("anh"):
        return 9999

    # MATCH DIREKTE PÅ ORIGINALSTRENG (IKKE fjern tegn!)
    m = re.match(r"^(\d+)", s)
    if m:
        return int(m.group(1))

    return 9999

# Lag k9_key2
df["k9_key2"] = df["k9_key"].apply(extract_k9_key2).astype("Int64")

# year som nullable int (ingen .0)
if "year" in df.columns:
    df["year"] = df["year"].astype("Int64")

if "age" in df.columns:
    df["age"] = df["age"].astype("Int64")

# Pad tall-delen til 4 tegn i k9_key3
def pad4(x):
    return f"{int(x):04d}" if pd.notna(x) else "9999"

df["k9_key2_str"] = df["k9_key2"].apply(pad4)
df["k9_key3"] = df["k9_key2_str"] + "|" + df["k9_key"].astype(str)

# ---- SORTER på k9_key3 ----
df = df.sort_values(by=["k9_key3"], ascending=True).reset_index(drop=True)

# Lag ny ID basert på sortert rekkefølge
df["id"] = range(1, len(df) + 1)

# Sjekk resultatet
print(df[["id", "k9_key", "k9_key2", "k9_key3"]].head(20))

# Lagre
df.to_csv("../data/mozart_catalog_rows_with_keys.csv", index=False)

print("✅ Ferdig: k9_key3 sortert, id regenerert, k9_key2=9999 når k9_key mangler.")


    id k9_key  k9_key2    k9_key3
0    1   1,01        1  0001|1,01
1    2   1,02        1  0001|1,02
2    3     1a        1    0001|1a
3    4     1b        1    0001|1b
4    5     1c        1    0001|1c
5    6     1d        1    0001|1d
6    7     1e        1    0001|1e
7    8     1f        1    0001|1f
8    9      2        2     0002|2
9   10      3        3     0003|3
10  11      4        4     0004|4
11  12      5        5     0005|5
12  13      6        6     0006|6
13  14      7        7     0007|7
14  15      8        8     0008|8
15  16      9        9     0009|9
16  17     9a        9    0009|9a
17  18     9b        9    0009|9b
18  19     10       10    0010|10
19  20     11       11    0011|11
✅ Ferdig: k9_key3 sortert, id regenerert, k9_key2=9999 når k9_key mangler.
